# ESM

In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [3]:
from scr.utils import pickle_load

In [4]:
df = pickle_load("data/proeng/gb1/two_vs_rest.pkl")

In [5]:
df.head()

,sequence,target,set,validation,mut_name,mut_numb
0,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYD...,1.000000,train,NaN,parent,0
1,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGIDGEWTYD...,1.445905,train,NaN,V39I,1
2,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGLDGEWTYD...,1.690164,train,NaN,V39L,1
3,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGMDGEWTYD...,1.170550,train,NaN,V39M,1
4,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVAGEWTYD...,2.401243,train,NaN,D40A,1


In [6]:
df_train = df.loc[(df["set"] == "train") & (df["validation"] != True)]
df_val = df.loc[(df["set"] == "train") & (df["validation"] == True)]
df_test = df.loc[(df["set"] == "test")]

len(df_train), len(df_val), len(df_test), len(df)

(381, 43, 8309, 8733)

In [7]:
from scr.encoding.encoding_classes import ESMEncoder

In [9]:
import torch

In [25]:
t = torch.tensor([[[1.0,2,3, 0], [4,5,6, 0]], [[7.0, 9, 0, 0], [10, 12, 0, 0]], [[7.0, 9, 8 , 10], [10, 12, 11, 13]]]) # [batch_size, seq_len, embed_dim] -> [batch_size, embed_dim]
t, t.shape

(tensor([[[ 1.,  2.,  3.,  0.],
          [ 4.,  5.,  6.,  0.]],
 
         [[ 7.,  9.,  0.,  0.],
          [10., 12.,  0.,  0.]],
 
         [[ 7.,  9.,  8., 10.],
          [10., 12., 11., 13.]]]),
 torch.Size([3, 2, 4]))

In [26]:
t.mean(axis=1), t.mean(axis=1).shape

(tensor([[ 2.5000,  3.5000,  4.5000,  0.0000],
         [ 8.5000, 10.5000,  0.0000,  0.0000],
         [ 8.5000, 10.5000,  9.5000, 11.5000]]),
 torch.Size([3, 4]))

In [28]:
sequence_representations = []
for rep, rep_len in zip(t, [3,2,4]):
    print(rep)
    print(rep, rep[rep_len].mean(0))
    sequence_representations.append(rep[rep_len].mean(0))


tensor([[1., 2., 3., 0.],
        [4., 5., 6., 0.]])


IndexError: index 3 is out of bounds for dimension 0 with size 2

In [37]:
import torch
import esm

# Load ESM-2 model
model, alphabet = esm.pretrained.esm1_t6_43M_UR50S()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

# Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
data = [
    ("protein1", "MKTVRQE"),
    ("protein2 with mask","KALTARQQEVFDLIRDISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[6], return_contacts=True)
token_representations = results["representations"][6]

torch.set_printoptions(profile="full")
print(token_representations) # prints the whole tensor
torch.set_printoptions(profile="default")
token_representations.shape

tensor([[[ 3.5937e-01,  1.1668e+00,  4.4240e-01,  3.2502e+00,  2.3139e+00,
           2.8567e+00, -7.0500e-01, -2.0718e+00,  1.8664e+00, -1.7033e+00,
           1.7560e+00,  3.2061e-02,  5.0992e+00,  1.5048e+00,  5.9560e-01,
           6.8678e-01, -1.8652e+00,  1.4871e+00, -7.6988e-01,  3.5425e-01,
           1.4201e-01,  1.7057e+00,  4.9822e-01, -1.4051e+00,  3.9267e-02,
           1.0820e+00,  1.6532e+00,  9.0789e-01,  3.7725e+00,  4.7640e+00,
          -2.0403e+00,  3.8933e+00,  1.2006e+00,  3.3623e+00,  3.0307e+00,
           6.5797e-01,  4.8499e-01, -2.2113e+00, -6.3981e-01, -1.5109e+00,
           3.0049e+00,  1.7447e+00, -1.2480e+00,  4.4015e-01,  7.3235e-01,
          -6.9826e-01,  1.3010e+00,  6.0572e-02, -2.0692e+00,  2.6114e+00,
           1.6996e+00,  2.8751e-01, -1.2849e-01,  1.5535e+00, -4.5245e-01,
          -4.9421e-01,  1.3032e+00, -1.4341e-01, -1.0185e+00, -7.6265e-01,
           8.2510e-01,  2.6799e-01, -9.8345e-02,  6.9144e-01, -9.1926e-01,
           5.0996e-02,  2

torch.Size([2, 71, 768])

In [38]:
sequence_representations = []
for i, (_, seq) in enumerate(data):
    print(seq)
    print(len(seq))
    print(token_representations[i, 1 : len(seq) + 1].shape)
    print(token_representations[i, 1 : len(seq) + 1].mean(0).shape)
    sequence_representations.append(token_representations[i, 1 : len(seq) + 1].mean(0))


MKTVRQE
7
torch.Size([7, 768])
torch.Size([768])
KALTARQQEVFDLIRDISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE
70
torch.Size([70, 768])
torch.Size([768])


In [42]:
torch.stack(sequence_representations).shape

torch.Size([2, 768])

In [43]:
torch.stack(sequence_representations) == token_representations.mean(axis=1)

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [44]:
import numpy as np

In [46]:
empty_np = np.empty((2,3))
empty_np

array([[4.6467916e-310, 0.0000000e+000, 0.0000000e+000],
       [0.0000000e+000, 0.0000000e+000, 0.0000000e+000]])

In [48]:
empty_np[0] = np.array([2,3,4])
empty_np

array([[2., 3., 4.],
       [0., 0., 0.]])

In [49]:
"""Add encoding classes with class methods"""

from __future__ import annotations

from abc import ABC, abstractmethod
from typing import Collection
from collections import Iterable, Sequence

import numpy as np
from tqdm import tqdm

import torch
from torch.nn.init import xavier_uniform_, xavier_normal_
from sequence_models.pretrained import load_model_and_alphabet

from scr.params.aa import AA_NUMB, AA_TO_IND
from scr.params.emb import TRANSFORMER_INFO, CARP_INFO
from scr.params.sys import DEVICE


class AbstractEncoder(ABC):
    """
    An abstract encoder class to fill in for different kinds of encoders

    All encoders will have an "encode" function
    """

    def __init__(
        self,
        encoder_name: str = "",
        reset_param: bool = False,
        resample_param: bool = False,
    ):

        """
        Args:
        - encoder_name: str, the name of the encoder, default empty for onehot
        - reset_param: bool = False, if update the full model to xavier_uniform_
        - resample_param: bool = False, if update the full model to xavier_normal_
        """

        self._encoder_name = encoder_name

        assert reset_param * resample_param != 1, "Choose reset OR resample param"

        self._reset_param = reset_param
        self._resample_param = resample_param

    def reset_resample_param(self, model: torch.nn.Module):
        """
        Initiate parameters in the PyTorch model. Following:
        https://pytorch.org/docs/stable/_modules/torch/nn/modules/transformer.html#Transformer

        Args:
        - model: torch.nn.Module, the input model

        Returns:
        - torch.nn.Module, the model with all params set with xavier_uniform
        """
        if self._reset_param:
            print(f"Reinit params for {self._encoder_name} ...")

            for p in model.parameters():
                if p.dim() > 1:
                    xavier_uniform_(p)

        elif self._resample_param:
            print(f"Resample params for {self._encoder_name} ...")

            for p in model.parameters():
                if p.dim() > 1:
                    xavier_normal_(p)

        return model

    def encode(
        self,
        mut_seqs: Sequence[str] | str,
        batch_size: int = 0,
        flatten_emb: bool | str = False,
        mut_names: Sequence[str] | str | None = None,
    ) -> Iterable[np.ndarray]:
        """
        A function takes a list of sequences to yield a batch of encoded elements

        Args:
        - mut_seqs: list of str or str, mutant sequences of the same length
        - batch_size: int, set to 0 to encode all in a single batch
        - flatten_emb: bool or str, if and how (one of ["max", "mean"]) to flatten the embedding
        - mut_names: list of str or str or None, mutant names

        Returns:
        - generator: dict with layer number as keys and
            encoded flattened sequence with or without labels as value
        """

        if isinstance(mut_seqs, str):
            mut_seqs = [mut_seqs]

        # If the batch size is 0, then encode all at once in a single batch
        if batch_size == 0:
            yield self._encode_batch(
                mut_seqs=mut_seqs, flatten_emb=flatten_emb, mut_names=mut_names
            )

        # Otherwise, yield chunks of encoded sequence
        else:

            for i in tqdm(range(0, len(mut_seqs), batch_size)):

                # figure out what mut_names to feed in
                if mut_names is None:
                    mut_name_batch = mut_names
                else:
                    mut_name_batch = mut_names[i : i + batch_size]

                yield self._encode_batch(
                    mut_seqs=mut_seqs[i : i + batch_size],
                    flatten_emb=flatten_emb,
                    mut_names=mut_name_batch,
                )

    def flatten_encode(
        self,
        encoded_mut_seqs: np.ndarray,
        flatten_emb: bool | str,
        mut_seqs: Sequence[str] | str,
    ) -> np.ndarray:
        """
        Flatten the embedding or just return the encoded mutants.

        Args:
        - encoded_mut_seqs: np.ndarray, shape [batch_size, seq_len, embed_dim]
        - flatten_emb: bool or str, if and how (one of ["max", "mean"]) to flatten the embedding
            - True -> shape [batch_size, seq_len * embed_dim]
            - "max" or "mean" -> shape [batch_size, embed_dim]
            - False or everything else -> [batch_size, seq_len, embed_dim]

        Returns:
        - np.ndarray, shape depends on flatten_emb parameter
        """

        assert (
            encoded_mut_seqs.shape[-1] == self._embed_dim
        ), f"encode last dim {encoded_mut_seqs.shape[-1]} != embed dim {self._embed_dim}"

        if flatten_emb in [True, "flatten", "flattened", ""]:
            # shape [batch_size, seq_len * embed_dim]
            return encoded_mut_seqs.reshape(encoded_mut_seqs.shape[0], -1)

        elif isinstance(flatten_emb, str):
            # init out put seq_reps should be in dim [batch_size, embed_dim]
            seq_reps = np.empty((encoded_mut_seqs.shape[0], self._embed_dim))
            for i, encoded_mut_seq in enumerate(encoded_mut_seqs):
                if flatten_emb == "mean":
                    seq_reps[i] = encoded_mut_seq[: len(mut_seqs[i])].mean(0)
                elif flatten_emb == "max":
                    seq_reps[i] = encoded_mut_seq[: len(mut_seqs[i])].max(0)

            return seq_reps

        else:
            # print("No embedding flattening")
            # [batch_size, seq_len, embed_dim]
            return encoded_mut_seqs

    @abstractmethod
    def _encode_batch(
        mut_seqs: Sequence[str] | str,
        flatten_emb: bool | str,
        mut_names: Sequence[str] | str | None = None,
    ) -> np.ndarray:
        """
        Encode a single batch of mut_seqs
        """
        pass

    @property
    def embed_dim(self) -> int:
        """The dim of the embedding"""
        return self._embed_dim

    @property
    def max_emb_layer(self) -> int:
        """The max layer nubmer of the embedding"""
        return self._max_emb_layer

    @property
    def include_input_layer(self) -> bool:
        """If include the input layer when counting the max layer number"""
        return self._include_input_layer

    @property
    def total_emb_layer(self) -> int:
        """Total embedding layer number"""
        return self._max_emb_layer + self._include_input_layer

    @property
    def encoder_name(self) -> str:
        """The name of the encoding method"""
        return self._encoder_name


class OnehotEncoder(AbstractEncoder):
    """
    Build a onehot encoder
    """

    def __init__(
        self,
        max_seq_len: int,
        encoder_name: str = "",
        reset_param: bool = False,
        resample_param: bool = False,
    ):
        """
        Args
        - encoder_name: str, the name of the encoder, one of the keys of CARP_INFO
        - max_seq_len: int, the longest sequence length
        - reset_param: bool = False, if update the full model to xavier_uniform_
        - resample_param: bool = False, if update the full model to xavier_normal_
        """
        super().__init__(encoder_name, reset_param, resample_param)

        self.max_seq_len = max_seq_len

        if encoder_name not in (TRANSFORMER_INFO.keys() and CARP_INFO.keys()):
            self._encoder_name = "onehot"
            self._embed_dim, self._max_emb_layer = AA_NUMB, 0
            self._include_input_layer = True

        # load model from torch.hub
        print(
            f"Generating {self._encoder_name} upto {self._max_emb_layer} layer embedding ..."
        )

        if reset_param or resample_param:
            self._reset_param = False
            self._resample_param = False
            print(
                f"Onehot encoding reset or resample param not allowed. /n \
                    Setting both to {self._reset_param} ..."
            )

    def _encode_batch(
        self,
        mut_seqs: Sequence[str] | str,
        flatten_emb: bool | str,
        mut_names: Sequence[str] | str | None = None,
    ) -> np.ndarray:

        encoded_mut_seqs = []

        for mut_seq in mut_seqs:
            # padding: (top, bottom), (left, right)
            encoded_mut_seqs.append(
                np.pad(
                    np.array(np.eye(AA_NUMB)[[AA_TO_IND[aa] for aa in mut_seq]]),
                    pad_width=((0, self.max_seq_len - len(mut_seq)), (0, 0)),
                )
            )

        return {
            0: self.flatten_encode(
                encoded_mut_seqs=np.array(encoded_mut_seqs),
                flatten_emb=flatten_emb,
                mut_seqs=mut_seqs,
            )
        }


class ESMEncoder(AbstractEncoder):
    """
    Build an ESM encoder
    """

    def __init__(
        self,
        encoder_name: str,
        reset_param: bool = False,
        resample_param: bool = False,
        iftrimCLS: bool = True,
        iftrimEOS: bool = True,
    ):
        """
        Args
        - encoder_name: str, the name of the encoder, one of the keys of TRANSFORMER_INFO
        - reset_param: bool = False, if update the full model to xavier_uniform_
        - resample_param: bool = False, if update the full model to xavier_normal_
        - iftrimCLS: bool, whether to trim the first classifification token
        - iftrimEOS: bool, whether to trim the end of sequence token, if exists
        """

        super().__init__(encoder_name, reset_param, resample_param)

        self._iftrimCLS = iftrimCLS
        self._iftrimEOS = iftrimEOS

        # get transformer dim and layer info
        self._embed_dim, self._max_emb_layer, _ = TRANSFORMER_INFO[self._encoder_name]

        # esm has the input representation
        self._include_input_layer = True

        # load model from torch.hub
        print(
            f"Generating {self._encoder_name} upto {self._max_emb_layer} layer embedding ..."
        )

        self.model, self.alphabet = torch.hub.load(
            "facebookresearch/esm:main", model=self._encoder_name
        )
        self.batch_converter = self.alphabet.get_batch_converter()

        # if reset or resample weights
        self.model = self.reset_resample_param(model=self.model)

        # set model to eval mode
        self.model.eval()
        self.model.to(DEVICE)

        expected_num_layers = int(self._encoder_name.split("_")[-3][1:])
        assert (
            expected_num_layers == self._max_emb_layer
        ), "Wrong ESM model name or layer"

    def _encode_batch(
        self,
        mut_seqs: Sequence[str] | str,
        flatten_emb: bool | str,
        mut_names: Sequence[str] | str | None = None,
    ) -> np.ndarray:
        """
        Encodes a batch of mutant sequences.

        Args:
        - mut_seqs: list of str or str, mutant sequences of the same length
        - flatten_emb: bool or str, if and how (one of ["max", "mean"]) to flatten the embedding
        - mut_names: list of str or str or None, mutant names

        Returns:
        - np.ndarray or a tuple(np.ndarray, list[str]) where the list is batch_labels
        """

        if isinstance(mut_names, str):
            mut_names = [mut_names]

        # pair the mut_names and mut_seqs
        if mut_names is not None:
            assert len(mut_names) == len(
                mut_seqs
            ), "mutant_name and mut_seqs different length"
            mut_seqs = [(n, m) for (n, m) in zip(mut_names, mut_seqs)]
        else:
            mut_seqs = [("", m) for m in mut_seqs]

        # convert raw mutant sequences to tokens
        batch_labels, _, batch_tokens = self.batch_converter(mut_seqs)
        batch_tokens = batch_tokens.to(DEVICE)

        # Turn off gradients and pass the batch through
        with torch.no_grad():
            # shape [batch_size, seq_len + pad, embed_dim]
            """if batch_tokens.shape[1] > TRANSFORMER_MAX_SEQ_LEN:
            print(f"Sequence exceeds {TRANSFORMER_MAX_SEQ_LEN}, taking the beginning and the end")
            batch_tokens = batch_tokens[:, :TRANSFORMER_MAX_SEQ_LEN]"""

            dict_encoded_mut_seqs = self.model(
                batch_tokens, repr_layers=list(range(self._max_emb_layer + 1))
            )["representations"]

        for layer, encoded_mut_seqs in dict_encoded_mut_seqs.items():

            encoded_mut_seqs = encoded_mut_seqs.cpu().numpy()
            # https://github.com/facebookresearch/esm/blob/main/esm/data.py
            # from_architecture

            # trim off initial classification token [CLS]
            # both "ESM-1" and "ESM-1b" have prepend_bos = True
            if self._iftrimCLS and self._encoder_name.split("_")[0] in [
                "esm1",
                "esm1b",
            ]:
                encoded_mut_seqs = encoded_mut_seqs[:, 1:, :]

            # trim off end-of-sequence token [EOS]
            # only "ESM-1b" has append_eos = True
            if self._iftrimEOS and self._encoder_name.split("_")[0] == "esm1b":
                encoded_mut_seqs = encoded_mut_seqs[:, :-1, :]

            if mut_names is not None:
                dict_encoded_mut_seqs[layer] = (
                    self.flatten_encode(
                        encoded_mut_seqs=encoded_mut_seqs,
                        flatten_emb=flatten_emb,
                        mut_seqs=mut_seqs,
                    ),
                    batch_labels,
                )
            else:
                dict_encoded_mut_seqs[layer] = self.flatten_encode(
                    encoded_mut_seqs=encoded_mut_seqs,
                    flatten_emb=flatten_emb,
                    mut_seqs=mut_seqs,
                )

        return dict_encoded_mut_seqs


class CARPEncoder(AbstractEncoder):
    """
    Build a CARP encoder
    """

    def __init__(
        self,
        encoder_name: str,
        reset_param: bool = False,
        resample_param: bool = False,
    ):
        """
        Args
        - encoder_name: str, the name of the encoder, one of the keys of CARP_INFO
        - reset_param: bool = False, if update the full model to xavier_uniform_
        - resample_param: bool = False, if update the full model to xavier_normal_
        """

        super().__init__(encoder_name, reset_param, resample_param)

        self.model, self.collater = load_model_and_alphabet(self._encoder_name)

        # if reset or resample weights
        self.model = self.reset_resample_param(model=self.model)

        # set model to eval mode
        self.model.eval()
        self.model.to(DEVICE)

        self._embed_dim, self._max_emb_layer = CARP_INFO[self._encoder_name]

        # carp does not have the input representation
        self._include_input_layer = False

        # load model from torch.hub
        print(
            f"Generating {self._encoder_name} upto {self._max_emb_layer} layer embedding ..."
        )

    def _encode_batch(
        self,
        mut_seqs: Sequence[str] | str,
        flatten_emb: bool | str,
        mut_names: Sequence[str] | str | None = None,
    ) -> np.ndarray:
        """
        Encodes a batch of mutant sequences.

        Args:
        - mut_seqs: list of str or str, mutant sequences of the same length
        - flatten_emb: bool or str, if and how (one of ["max", "mean"]) to flatten the embedding
        - mut_names: list of str or str or None, mutant names

        Returns:
        - np.ndarray or a tuple(np.ndarray, list[str]) where the list is batch_labels
        """

        mut_seqs = [[m] for m in mut_seqs]

        x = self.collater(mut_seqs)[0]

        # alternatively check out the article called:
        # The One PyTorch Trick Which You Should Know
        # How hooks can improve your workflow significantly

        activation = {}

        def get_activation(name):
            def hook(model, input, output):
                activation[name] = output.detach()

            return hook

        # convert raw mutant sequences to tokens
        for layer_numb in list(range(self._max_emb_layer)):
            self.model.model.embedder.layers[layer_numb].register_forward_hook(
                get_activation(layer_numb)
            )

        rep = self.model(x)

        for layer_numb, encoded_mut_seqs in activation.items():
            activation[layer_numb] = self.flatten_encode(
                encoded_mut_seqs=encoded_mut_seqs.cpu().numpy(),
                flatten_emb=flatten_emb,
                mut_seqs=mut_seqs,
            )

        return activation


def get_emb_info(encoder_name: str) -> Collection(str, AbstractEncoder, int):

    """
    A function return processed encoder_name and total_emb_layer

    Args:
    - encoder_name: str, input encoder_name

    Returns:
    - encoder_name: str, change anything not a transformer or carp encoder to onehot
    - encoder_class: AbstractEncoder, encoder class
    - total_emb_layer: int, number of embedding layers
    """

    if encoder_name in TRANSFORMER_INFO.keys():
        total_emb_layer = TRANSFORMER_INFO[encoder_name][1] + 1
        encoder_class = ESMEncoder
    elif encoder_name in CARP_INFO.keys():
        total_emb_layer = CARP_INFO[encoder_name][1]
        encoder_class = CARPEncoder
    else:
        # for onehot
        encoder_name = "onehot"
        encoder_class = OnehotEncoder
        total_emb_layer = 1

    return encoder_name, encoder_class, total_emb_layer

In [55]:
no_flat_encoder = ESMEncoder(
    encoder_name="esm1b_t33_650M_UR50S"
).encode(mut_seqs=list(df_val.sequence), mut_names=list(df_val.mut_name))
one_emb = next(no_flat_encoder)
one_emb

Generating esm1b_t33_650M_UR50S upto 33 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


{0: (array([[[ 0.27928993, -0.00919111,  0.05307703, ...,  0.0052783 ,
           -0.08536894,  0.2144556 ],
          [ 0.8682009 ,  0.01390769, -0.03625386, ..., -0.07747607,
           -0.06879894,  0.10098854],
          [ 0.9904541 ,  0.02066673,  0.04346693, ..., -0.04951385,
            0.04328922, -0.09477672],
          ...,
          [ 0.01445956,  0.15244195,  0.13133049, ...,  0.00364488,
            0.50209504,  0.20528588],
          [ 0.1433187 ,  0.08901688,  0.06859886, ..., -0.04608465,
            0.14192118,  0.12139929],
          [-0.07774346,  0.03318178,  0.14130217, ..., -0.08603121,
            0.30741137,  0.01056057]],
  
         [[ 0.27928993, -0.00919111,  0.05307703, ...,  0.0052783 ,
           -0.08536894,  0.2144556 ],
          [ 0.8682009 ,  0.01390769, -0.03625386, ..., -0.07747607,
           -0.06879894,  0.10098854],
          [ 0.9904541 ,  0.02066673,  0.04346693, ..., -0.04951385,
            0.04328922, -0.09477672],
          ...,
         

In [58]:
one_emb[0], one_emb[0][0].shape

((array([[[ 0.27928993, -0.00919111,  0.05307703, ...,  0.0052783 ,
           -0.08536894,  0.2144556 ],
          [ 0.8682009 ,  0.01390769, -0.03625386, ..., -0.07747607,
           -0.06879894,  0.10098854],
          [ 0.9904541 ,  0.02066673,  0.04346693, ..., -0.04951385,
            0.04328922, -0.09477672],
          ...,
          [ 0.01445956,  0.15244195,  0.13133049, ...,  0.00364488,
            0.50209504,  0.20528588],
          [ 0.1433187 ,  0.08901688,  0.06859886, ..., -0.04608465,
            0.14192118,  0.12139929],
          [-0.07774346,  0.03318178,  0.14130217, ..., -0.08603121,
            0.30741137,  0.01056057]],
  
         [[ 0.27928993, -0.00919111,  0.05307703, ...,  0.0052783 ,
           -0.08536894,  0.2144556 ],
          [ 0.8682009 ,  0.01390769, -0.03625386, ..., -0.07747607,
           -0.06879894,  0.10098854],
          [ 0.9904541 ,  0.02066673,  0.04346693, ..., -0.04951385,
            0.04328922, -0.09477672],
          ...,
          [ 

In [57]:
mean_flat_encoder = ESMEncoder(
    encoder_name="esm1b_t33_650M_UR50S",
).encode(mut_seqs=list(df_val.sequence), mut_names=list(df_val.mut_name),flatten_emb="mean")
one_mean_emb = next(mean_flat_encoder)
one_mean_emb

Generating esm1b_t33_650M_UR50S upto 33 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


{0: (array([[-0.17739098, -0.00067187,  0.01667726, ...,  0.00655907,
           0.03614971,  0.09654725],
         [-0.1747657 , -0.00065019,  0.01660449, ...,  0.00632377,
           0.03780067,  0.09638622],
         [-0.1749173 , -0.00061091,  0.01679106, ...,  0.00663596,
           0.03946506,  0.09655192],
         ...,
         [-0.177018  , -0.00064966,  0.0166867 , ...,  0.00618463,
           0.03673363,  0.09675171],
         [-0.1757744 ,  0.00030468,  0.01655487, ...,  0.00629132,
           0.03641493,  0.09631334],
         [-0.17786627, -0.00057332,  0.01687178, ...,  0.00618563,
           0.03663645,  0.09616786]], dtype=float32),
  ['D40N',
   'V39E:D40Y',
   'V39F:V54G',
   'V39K:D40I',
   'V39L:D40A',
   'V39L:D40I',
   'V39L:D40R',
   'V39L:D40S',
   'V39L:D40V',
   'V39M:V54L',
   'V39M:D40Q',
   'V39M:D40T',
   'V39Q:D40H',
   'V39Q:D40S',
   'V39R:D40R',
   'V39R:D40Y',
   'V39T:G41K',
   'V39T:D40K',
   'V39T:D40T',
   'D40A:V54N',
   'D40A:V54T',
   'D40A:V5

In [59]:
one_mean_emb[0], one_mean_emb[0][0].shape

((array([[-0.17739098, -0.00067187,  0.01667726, ...,  0.00655907,
           0.03614971,  0.09654725],
         [-0.1747657 , -0.00065019,  0.01660449, ...,  0.00632377,
           0.03780067,  0.09638622],
         [-0.1749173 , -0.00061091,  0.01679106, ...,  0.00663596,
           0.03946506,  0.09655192],
         ...,
         [-0.177018  , -0.00064966,  0.0166867 , ...,  0.00618463,
           0.03673363,  0.09675171],
         [-0.1757744 ,  0.00030468,  0.01655487, ...,  0.00629132,
           0.03641493,  0.09631334],
         [-0.17786627, -0.00057332,  0.01687178, ...,  0.00618563,
           0.03663645,  0.09616786]], dtype=float32),
  ['D40N',
   'V39E:D40Y',
   'V39F:V54G',
   'V39K:D40I',
   'V39L:D40A',
   'V39L:D40I',
   'V39L:D40R',
   'V39L:D40S',
   'V39L:D40V',
   'V39M:V54L',
   'V39M:D40Q',
   'V39M:D40T',
   'V39Q:D40H',
   'V39Q:D40S',
   'V39R:D40R',
   'V39R:D40Y',
   'V39T:G41K',
   'V39T:D40K',
   'V39T:D40T',
   'D40A:V54N',
   'D40A:V54T',
   'D40A:V54W'

Bad pipe message: %s [b"\x9d\xf5E\xc5&'F\xca\xb0\x7f,\x84@\xda:\xe6\xa7\xb8\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04"]
Bad pipe message: %s [b'', b'']
Bad pipe message: %s [b'\\,=\x001E\x1e\xee\xa6\x04J\xc1', b"\xcd\x8f\xab\xb1\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\

In [10]:
mean_batch_encoder = ESMEncoder(
    encoder_name="esm1b_t33_650M_UR50S",
    embed_layer=0,
).encode(mut_seqs=list(df_val.sequence), mut_names=list(df_val.mut_name), batch_size=4, flatten_emb="mean")
one_mean_emb = next(mean_batch_encoder)

Loading esm1b_t33_650M_UR50S using 0 layer embedding


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main
  0%|          | 0/11 [00:00<?, ?it/s]

In [11]:
one_mean_emb[0].shape, len(one_mean_emb[1])

((4, 1280), 4)

In [14]:
one_mean_emb[1], df_val.mut_name[:4]

(['D40N', 'V39E:D40Y', 'V39F:V54G', 'V39K:D40I'],
 20          D40N
 56     V39E:D40Y
 60     V39F:V54G
 117    V39K:D40I
 Name: mut_name, dtype: object)

In [3]:
import torch

# Load ESM-1b model
original_model, alphabet = torch.hub.load(
            "facebookresearch/esm:main", 
            # model="esm1_t6_43M_UR50S"
            model="esm1b_t33_650M_UR50S"
        )
batch_converter = alphabet.get_batch_converter()
data = [
    ("protein1", "MSGEEEKAADFYVRYYVGHKGKFGHEFLEFEFRPNGSLRYANNSNYKNDTMIRKEATVSESVLSELKRIIEDSEIMQEDDDNWPEPDKIGRQELEILYKNEHISFTTGKIGALADVNNSKDPDGLRSFYYLVQDLKCLVFSLIGLHFKIKPI"),
    ("2", "VSDGMTGSNPVSPASSSSPDSSGAGGISPQHIAQDSSLDGPPGPQDGTTVPLEGFSLSHAADLVNRGQKWEKSHAEIAEQAKHEAEIETRIAELRKEGFWSLKRLPKVPEPPRPKGHWDYLCEEMQWLSADFAQERRWKRGVARKVVRMVIRHHEEQRQKEERARREEQAKLRRIASTMAKDVRQFWSNVEKVVQFKQQSRLEEKRKKALDLHLDFIVGQTEKYSDLLSQSLNQPPASSKAGSSPCLGSSSAASSPPPPVSRLDDEDGDFQPQEEEEEDDEETIEVEEQQEGNDAETQRREIELLRHEGELPLEELLRSLPPQLLGGPFSPSQTPSHDSDTQDGPEENIEEEPSQDLEVHPPSSAVTQCNKQRWHPDEDDEEFTANEDEAEDEEDTIAAEEQLEGEVDHAMELSELAREGELSMEELLQQYAGAYACDASAPASGDSEDEDEVEANSSDGELEETVEEAAQEDSSSQSDSAEECSEDEEDEHSEEEMSGSSQSEESESDESEDAQSQSQADEEQDDEGEDDDVDDDDDDGFGVEYLLARDDERSEVDGGSGPPTPGPTTTLGPKKEITDIAAAAESLQPKGYTLATTQVKTPIPLLLRGQLREYQHIGLDWLVTMYEKKLNGILADEMGLGKTIQTISLLAHLACEKGNWGPHLIIVPTSVMLNWEMELKRWCPSFKILTYYGAQKERKLKRQGWTKPNAFHVCITSYKLVLQDHQAFRRKNWRYLILDEAQNIKNFKSQRWQSLLNFNSQRRLLLTGTPLQNSLMELWSLMHFLMPHVFQSHREFKEWFSNPLTGMIEGSQEYNEGLVKRLHKVLRPFLLRRVKVDVEKQMPKKYEHVIRCRLSKRQRCLYDDFMAQTTTKETLATGHFMSVINILMQLRKVCNHPNLFDPRPVTSPFITPGICFSTASLVLRATEVHPLQRIDMGRFDLIGLEGRVSRYEADTFLPRHRLSHRILLEIATAPDPPPRPKPVKMKVNRMLQPVPKQEGRTVVVVNSPRPPLGPVPVRPPPGPEVSAQPALGPVPPVLPAPLMVSASPAGTPVVPASRPPGPVLLSPLQPNTGPLPQAGEVVSIGQLASLAQRPVTSAGGSKPLTFQIQGNKLTLTGAQVRQLAVGQPRPLQMPPSMVNNTGVVKIVVRQAPRDGLTPVPPLAPAPRPSSSGLPAVLTSRPTLTPSRLSSPTLGTTRTPIPTPTLVRPLLKLVHSPSPEVSASAPGAAPMTISSSLHMPSSLPGPASSPMTIHNSSPLSSPVSSTVSVPVSSSLPISVPTTLPVPTSAPLPIPITAPLPVSDLGPALLTSVTPALAVVSAASGPPLASAGVSPSMSALALGLATTPALSPPQTSGHSLLLATTSSHVPGLNSAVAPACSPVLVSASALTSPFPVSPSSAPAQASLLAAAPSTSQALATSLAPMAASQTAILGPSPTPPLAPLPVLAASQTPLPAMTPPSMSGTPLPSSSLVSAPTSVLAPSSTQTLVSTPVSSPLSSLASTQTLTLAPTLAPTLGGLSPSQTHSLGTGSPQGPFPIQTLSLTPASSLVPTPAQTLSLAPGGPVGPTQTLSLAPVPPLPPSSPVGPAPGHTLTLAPAASSASLLVPTSVQTLTLSPAPVPVPTLDLSATQTLALAPALTQAPASQASSLVSASGAASLPVTMVNRLPVPKDEPETLTLRSGPPSPLPTATSFSGPRPRRQPPPPPRSPFYLDSLEEKRKQQRSERLDRIFQLSEAHGALAPVYGTEVLDFCTLPQPVASPIGPRSPGPSHPIFWTYTEAARRAVLFPQQRLEQLSEIIERFIFVMPPVEAPPPSLHACHPPPWLAPHQAAFQEQLACELWPRARPLHRIVCNMRTQFPDLRLIQYDCGKLQTLAVLLRQLKAEGHRVLIFTQMTRMLDVLEQFLTYHGHLYLRLDGSTRVEQRQALMERFNADKRIFCFILSTRSGGVGVNLTGADTVVFYDSDWNPTMDAQAQDRCHRIGQTRDVHIYRLISERTVEENILKKANQKRMLGDMAIEGGNFTTAYFKQQTIRELFDMPLEEPPGSSVSSVPEDEEEAVASKQTHILEQALCRAEDEEDIRAATQAKAEQVAELAEFNENDGFPAGEGEEANRPGPGAEDEEMSRAEQEIAALVEQLTPIERYAMKFLEASLEEVSREELKQAEEQVEAARKDLDQAKEEVFRLPHEEEEGPGAGDEMSCGSSGASHRRSKKIRAPERLGTRVSERLRGARAETQGANHTPVTSSHHTCNTFVPPQCSPVRERIPRPVPRPRPPPPPPAPPPPPAPPASPSPAPPPPPAPASPAPPAPPASPPAASPSPAPLPPAASVASAITSALATAPIPISAPNPVTILPVHILPTPPLLQLPPSSSPASTPPSACTPLPTCTPLQAQTSVLTSSSPAPLDSLSVPISPGVTNLPLDSGEVGLCAQELAPVESLELPGMTNSEASLASLVSPKDLLPVAVDILPESEKTLPCIPAPSSTLEAESIPNGQQQEVPGPVEGTNTTLLPGSEELTMSLNESNGLELPPSAASAEPLDELLETHRNSGETREVQIPISNSEKPQELVRAEVTAPSTSSSATSSPEGPLPARPPRRRTSADVEIRGQGSGRTGQLPGPKVLRKLPGRLVTVVEEKELVKRRRHRAPASTLVPGVSEVSASPRSPSIHSMSGPESSPPSSGPCEAAPTSSLSTPTQSTFIACRHNELGTTTGGSPENGEGTELAITPPAVKRRRGRPPKKNRSPADAGRGVDEVPSSISKGKTNGVDPVPGPETVTVAEPDLRPQFVPGPQPPGPQPVHRPEPTILSPVEKRRRGRPPKARDLPIPGTISSPGGGNLESRTQSLPFPAPLTPLLACPTAVTNTVTTVTISTSPPKRKRGRPPKNPPSPRPSQHPVLDPDSSSILESCGLGVQRQPLGQAESEGSSSDEDGSRPLTRLARLRLEAEGMRGRKSEGSMVMAVIQDDLDLADSGPSGLELTPPVVSLTPKFRSTRLRPGSLVPPLETEKMPRKRSGAPVGSSGMAKRGRLQPPSPLGPEGSVEESEVEASGDEEEGDGTPRRRTGPRRLVGTTNQGDQRILRSSAPPHLCIPTISHRGRKAKT")
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

original_model.eval()
with torch.no_grad():
    original_results = original_model(batch_tokens, repr_layers=[6], return_contacts=False)
original_reps = original_results["representations"][6].numpy().mean(axis=1).flatten()

Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1b_t33_650M_UR50S.pt" to /home/t-fli/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm1b_t33_650M_UR50S-contact-regression.pt" to /home/t-fli/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S-contact-regression.pt


ValueError: Sequence length 3097 above maximum  sequence length of 1024

In [12]:
batch_tokens.shape

torch.Size([2, 3096])